In [1]:
import requests
from full_obj import body

# Testando o primeiro endpoint

In [2]:
url = 'http://127.0.0.1:8000/v1/'


Ele precisa de um body que é uma lista com as varios registros, veja o arquivo full_obj.py para mais detalhes

In [3]:
x = requests.post(url,json=body)

In [4]:
x.text

'{"volumetria":{"July":74,"August":72,"May":67,"June":63,"March":62,"January":58,"February":55,"April":49},"ROC-AUC":0.5751748251748252}'

# Testando o segundo endpoint

Para os proximos testes:

vamos passar qual arquivo estamos comparando com o dataset de test.gz.  
Opcionalmente podemos escolher outra metrica(Mannwithneyu)

## Testando o dataset oot

In [5]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Foot.gz'

In [6]:
x = requests.post(url)


In [7]:
print(x.text)

{"estatistica de Kolmogorov-Smirnov":0.09172708533663676,"p":1.0236104206010673e-225}


Esse p-value é muito baixo então podemos rejeitar a hipotese nula, nesse caso as distribuições são diferentes

## Testando o dataset de teste

In [8]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Ftest.gz'

In [9]:
x = requests.post(url)

In [10]:
print(x.text)

{"estatistica de Kolmogorov-Smirnov":0.0,"p":1.0}


como era o esperado esse pvalue é muito alto(já que estamos testando a mesma distribuição com ela mesmo), nesse caso a hipotese nula não pode ser rejeitada e as distribuições são iguais

## Testando o dataset de train

In [11]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Ftrain.gz'


In [12]:
x = requests.post(url)

In [13]:
print(x.text)

{"estatistica de Kolmogorov-Smirnov":0.002759858953621075,"p":0.9605978662359891}


o pvalue é muito alto, nesse caso a hipotese nula não pode ser rejeitada e as distribuições provavelmente não são diferentes

# Testando a estatistica de Mannwithneyu

In [14]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Foot.gz&test=MAN'


Essa estatistica é bem similar ao KS, mas ela é mais famosa. Ambas são não parametricas e ambas podem ser usadas para testar duas distribuições não pareadas

In [15]:
x = requests.post(url)

In [16]:
print(x.text)

{"estatistica de mannwithneyu":1859523611.5,"p":1.7244757437036297e-111}


mesmo resultado comentado anteriormente para o teste KS

# Treinando um novo modelo

In [18]:
!pip install xgboost

In [32]:
import pandas as pd

from xgboost import XGBClassifier
import numpy as np

In [76]:
df_train = pd.read_csv("../datasets/credit_01/test.gz",compression='gzip')

/home/vtex/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [77]:
df_train.fillna(value=np.nan,inplace=True)

In [82]:
X = df_train.drop(['TARGET',"REF_DATE"],axis=1)
y = df_train["TARGET"]

In [83]:
print(X.dtypes)

VAR2       object
IDADE     float64
VAR4       object
VAR5       object
VAR6      float64
           ...   
VAR146    float64
VAR147      int64
VAR148     object
VAR149     object
ID          int64
Length: 149, dtype: object


In [86]:
for col in ['VAR2', 'VAR4', 'VAR5', 'VAR8', 'VAR9', 'VAR10', 'VAR31', 'VAR32', 'VAR33', 'VAR34', 'VAR35', 'VAR36', 'VAR38', 'VAR43', 'VAR45', 'VAR48', 'VAR49', 'VAR50', 'VAR51', 'VAR52', 'VAR53', 'VAR54', 'VAR55', 'VAR56', 'VAR57', 'VAR58', 'VAR59', 'VAR60', 'VAR61', 'VAR62', 'VAR63', 'VAR64', 'VAR65', 'VAR66', 'VAR67', 'VAR68', 'VAR69', 'VAR70', 'VAR71', 'VAR72', 'VAR73', 'VAR74', 'VAR75', 'VAR76', 'VAR77', 'VAR78', 'VAR79', 'VAR80', 'VAR81', 'VAR82', 'VAR83', 'VAR84', 'VAR85', 'VAR86', 'VAR87', 'VAR88', 'VAR89', 'VAR90', 'VAR91', 'VAR92', 'VAR93', 'VAR94', 'VAR95', 'VAR96', 'VAR97', 'VAR98', 'VAR99', 'VAR100', 'VAR101', 'VAR102', 'VAR103', 'VAR104', 'VAR105', 'VAR106', 'VAR107', 'VAR108', 'VAR109', 'VAR110', 'VAR111', 'VAR112', 'VAR113', 'VAR114', 'VAR115', 'VAR116', 'VAR117', 'VAR118', 'VAR119', 'VAR120', 'VAR121', 'VAR122', 'VAR123', 'VAR124', 'VAR125', 'VAR126', 'VAR127', 'VAR128', 'VAR129', 'VAR130', 'VAR131', 'VAR132', 'VAR133', 'VAR134', 'VAR135', 'VAR136', 'VAR137', 'VAR138', 'VAR139', 'VAR140', 'VAR142', 'VAR143', 'VAR144', 'VAR148', 'VAR149']:
    X[col] = X[col].astype('category')

In [87]:
%%time
xgb = XGBClassifier(tree_method="hist", enable_categorical=True)
xgb.fit(X, y)


CPU times: user 1min 32s, sys: 642 ms, total: 1min 32s
Wall time: 16.2 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [88]:
from sklearn.metrics import classification_report

In [89]:
df_test = pd.read_csv("../datasets/credit_01/test.gz",compression='gzip')

/home/vtex/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [90]:
df_test.fillna(value=np.nan,inplace=True)

In [94]:
X_test = df_test.drop(['TARGET',"REF_DATE"],axis=1)
y_test = df_test["TARGET"]

In [97]:
for col in ['VAR2', 'VAR4', 'VAR5', 'VAR8', 'VAR9', 'VAR10', 'VAR31', 'VAR32', 'VAR33', 'VAR34', 'VAR35', 'VAR36', 'VAR38', 'VAR43', 'VAR45', 'VAR48', 'VAR49', 'VAR50', 'VAR51', 'VAR52', 'VAR53', 'VAR54', 'VAR55', 'VAR56', 'VAR57', 'VAR58', 'VAR59', 'VAR60', 'VAR61', 'VAR62', 'VAR63', 'VAR64', 'VAR65', 'VAR66', 'VAR67', 'VAR68', 'VAR69', 'VAR70', 'VAR71', 'VAR72', 'VAR73', 'VAR74', 'VAR75', 'VAR76', 'VAR77', 'VAR78', 'VAR79', 'VAR80', 'VAR81', 'VAR82', 'VAR83', 'VAR84', 'VAR85', 'VAR86', 'VAR87', 'VAR88', 'VAR89', 'VAR90', 'VAR91', 'VAR92', 'VAR93', 'VAR94', 'VAR95', 'VAR96', 'VAR97', 'VAR98', 'VAR99', 'VAR100', 'VAR101', 'VAR102', 'VAR103', 'VAR104', 'VAR105', 'VAR106', 'VAR107', 'VAR108', 'VAR109', 'VAR110', 'VAR111', 'VAR112', 'VAR113', 'VAR114', 'VAR115', 'VAR116', 'VAR117', 'VAR118', 'VAR119', 'VAR120', 'VAR121', 'VAR122', 'VAR123', 'VAR124', 'VAR125', 'VAR126', 'VAR127', 'VAR128', 'VAR129', 'VAR130', 'VAR131', 'VAR132', 'VAR133', 'VAR134', 'VAR135', 'VAR136', 'VAR137', 'VAR138', 'VAR139', 'VAR140', 'VAR142', 'VAR143', 'VAR144', 'VAR148', 'VAR149']:
    X_test[col] = X_test[col].astype('category')

In [98]:
pred = xgb.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.95      0.34      0.50     10369
           1       0.85      0.99      0.92     39441

    accuracy                           0.86     49810
   macro avg       0.90      0.67      0.71     49810
weighted avg       0.87      0.86      0.83     49810



In [99]:
from sklearn.metrics import roc_auc_score

In [101]:
pred = xgb.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,pred))

0.9213929372695054
